## Experiment 1 - Memory efficiency of commSignal

Refs: [1] https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/

The goal of this experiment is to verify that the commSignal object is memory efficient. Inefficiencies may occur when the internal signal storage is copied everytime it is being used. To verify that the same array is being used the following functions are defined, courtesy of [1]

In [1]:
import numpy as np

def get_data_base(arr):
    """For a given NumPy array, find the base array
    that owns the actual data."""
    base = arr
    while isinstance(base.base, np.ndarray):
        base = base.base
    return base


def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

a = np.array(range(100))
print(arrays_share_data(a, a.copy()))
print(arrays_share_data(a, a[:1]))

False
True


In the constructor we expect the signal to be copied so as to preserve a copy within the object, the following result shows this is true.

In [3]:
class test:
    def __init__(self, sig):
        self.__sig = np.array(sig)
        print(arrays_share_data(self.__sig, sig))
        
test(a)

False


Next we test what happens when we return the signal, is it copied then aswell? We see that it isn't being copied again. This is what we want, everytime we return the signal we dont want it to be copied again and again, which will lead to unnecessary delays

In [5]:
class test:
    def __init__(self, sig):
        self.__sig = np.array(sig)
        print(arrays_share_data(self.__sig, self.getsig))
        print(arrays_share_data(self.getgetsig, self.getsig))
    @property
    def getsig(self):
        return self.__sig
    @property
    def getgetsig(self):
        return self.getsig
    
test(a)

True
True


### Conclusions
We conclude that when we return the signal array, it is not being copied again hence the object is memory efficient.